<a href="https://colab.research.google.com/github/OriolOriolOriol/Perseus/blob/main/Perseus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Importato le librerie necessari per creare il modello.
Ecco di seguito cosa fanno:


In [27]:
filename="/content/libro.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

Carichiamo  il libro  in memoria e convertire tutti i caratteri in minuscolo per ridurre il vocabolario che la rete deve imparare.

In [28]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)


{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, '_': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'i': 31, 'j': 32, 'l': 33, 'm': 34, 'n': 35, 'o': 36, 'p': 37, 'q': 38, 'r': 39, 's': 40, 't': 41, 'u': 42, 'v': 43, 'x': 44, 'y': 45, 'z': 46, '«': 47, '°': 48, '²': 49, '¹': 50, '»': 51, 'à': 52, 'è': 53, 'ê': 54, 'ì': 55, 'î': 56, 'ò': 57, 'ù': 58}


Ora che il libro è caricato, dobbiamo preparare i dati per la modellazione dalla rete neurale. Non possiamo modellare i caratteri direttamente, dobbiamo invece convertire i caratteri in numeri interi.
Prendi ogni carattere e le ordini in abse al suo codice ASCII. Successivamente si mappa ogni singolo carattere con un numero

In [29]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  120448
Total Vocab:  59


Ora che il libro è stato caricato e la mappatura preparata, possiamo riassumere il set di dati.

In [30]:
lunghezza_sequenza= 100
dataX=[]
dataY=[]
for i in range(0, n_chars - lunghezza_sequenza, 1):
  seq_in= raw_text[i:i + lunghezza_sequenza]
  seq_output= raw_text[i + lunghezza_sequenza]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_output])
n_patterns= len(dataX)
print("Totali Patterns: ", n_patterns)

Totali Patterns:  120348


Ora dobbiamo definire i dati di addestramento per la rete. C'è molta flessibilità nel modo in cui scegli di suddividere il testo ed esporlo alla rete durante l'allenamento.

Possiamo dividere il testo del libro in sottosequenze di 100 caratteri. Ciascun training patterns è composto da 100 fasi temporali di un carattere X seguito da un output di carattere Y.


Quando creiamo queste sequenze, facciamo scorrere questa finestra lungo l'intero libro un carattere alla volta, consentendo a ciascun carattere di apprendere dai 100 caratteri che lo hanno preceduto (tranne i primi 100 caratteri ovviamente).

Una volta suddivisi convertiamo i caratteri in numeri interi usando la funzione creata in precedenza.

L'esecuzione del codice fino a questo punto ci mostra che quando suddividiamo il set di dati in dati di addestramento affinché la rete apprenda che abbiamo poco meno di 150.000 modelli di addestramento. Questo ha senso escludendo i primi 100 caratteri, abbiamo un modello di addestramento per prevedere ciascuno dei caratteri rimanenti.


In [31]:
#X

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, lunghezza_sequenza, 1))
# normalize
X = X / float(n_vocab)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, lunghezza_sequenza, 1))
# normalize
X = X / float(n_vocab)

#Y
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Ora che abbiamo preparato i nostri dati di allenamento, dobbiamo trasformarli in modo che siano adatti per l'uso con Keras.

Per prima cosa dobbiamo trasformare l'elenco delle sequenze di input nella forma [campioni, fasi temporali, caratteristiche] attesi da una rete LSTM.

Successivamente è necessario ridimensionare gli interi nell'intervallo da 0 a 1 per rendere i modelli più facili da apprendere dalla rete LSTM che utilizza la funzione di attivazione del sigmoide per impostazione predefinita.

Infine, dobbiamo convertire i modelli di output (singoli caratteri convertiti in interi) in una codifica a caldo. Questo è così che possiamo configurare la rete per prevedere la probabilità di ciascuno dei 47 diversi caratteri nel vocabolario (una rappresentazione più semplice) piuttosto che cercare di costringerla a prevedere con precisione il carattere successivo. Ogni valore y viene convertito in un vettore sparse con una lunghezza di 47, pieno di zeri tranne con un 1 nella colonna per la lettera (numero intero) che il modello rappresenta.

Ad esempio, quando "n" (valore intero 31) è una codifica a caldo, appare come segue:

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0. 0.  0.  0.  0.  0.  0.  0.  0.]

In [32]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax')) #Softmax output shape
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=30, batch_size=256, callbacks=callbacks_list)

Epoch 1/20
941/941 [==============================] - 20s 13ms/step - loss: 3.0087

Epoch 00001: loss improved from inf to 2.95358, saving model to weights-improvement-01-2.9536.hdf5
Epoch 2/20
941/941 [==============================] - 12s 13ms/step - loss: 2.8340

Epoch 00002: loss improved from 2.95358 to 2.80814, saving model to weights-improvement-02-2.8081.hdf5
Epoch 3/20
941/941 [==============================] - 12s 13ms/step - loss: 2.7591

Epoch 00003: loss improved from 2.80814 to 2.74885, saving model to weights-improvement-03-2.7488.hdf5
Epoch 4/20
941/941 [==============================] - 12s 13ms/step - loss: 2.7211

Epoch 00004: loss improved from 2.74885 to 2.71788, saving model to weights-improvement-04-2.7179.hdf5
Epoch 5/20
941/941 [==============================] - 12s 13ms/step - loss: 2.6978

Epoch 00005: loss improved from 2.71788 to 2.68626, saving model to weights-improvement-05-2.6863.hdf5
Epoch 6/20
941/941 [==============================] - 12s 13ms/step -

Qui definiamo un singolo livello LSTM nascosto con 256 unità di memoria. La rete utilizza dropout con una probabilità di 20. Il livello di output è un livello Denso che utilizza la funzione di attivazione softmax per produrre una previsione di probabilità per ciascuno dei 47 caratteri compresi tra 0 e 1.

Il problema è in realtà un problema di classificazione di un singolo carattere con 47 classi e come tale è definito come l'ottimizzazione della perdita di log (entropia incrociata), qui utilizzando l'algoritmo di ottimizzazione ADAM per la velocità.



Non siamo interessati al modello più accurato (accuratezza della classificazione) del set di dati di addestramento. Questo sarebbe un modello che prevede perfettamente ogni personaggio nel set di dati di addestramento. Ci interessa invece una generalizzazione del dataset che minimizzi la funzione di perdita scelta. Cerchiamo un equilibrio tra generalizzazione e overfitting ma a corto di memorizzazione.


La rete è lenta da addestrare (circa 300 secondi per epoca su una GPU Nvidia K520). A causa della lentezza e dei nostri requisiti di ottimizzazione, utilizzeremo il checkpoint del modello per registrare tutti i pesi della rete da archiviare ogni volta che si osserva un miglioramento della perdita alla fine dell'epoca. Useremo il miglior set di pesi (perdita minima) per istanziare il nostro modello generativo nella sezione successiva.